In [1]:
import numpy as np
import random
# Run the fitness 
from oct2py import Oct2Py
from multiprocessing.pool import ThreadPool 
# Run multiple Zs as different processes
from multiprocessing import Pool
# to save the progress of the algorithms
import csv
from datetime import datetime
from os import mkdir

In [2]:
# -------- range of the variables ----------
V_S = 7.0                   # service speed [kn]
range_D     = [0.5, 0.8]
range_AEdAO = [0.3, 1.05]
range_PdD   = [0.5, 1.4]
range_Z     = [2, 7]

# Define the lower and upper bounds for each variable
lower_bounds = [range_D[0], range_AEdAO[0], range_PdD[0]]
upper_bounds = [range_D[1], range_AEdAO[1], range_PdD[1]]

In [3]:
   # size of step from each variable
LIST_SIZE = {'D':     int((range_D[1]-range_D[0])        / 0.1),
             'AEdAO': int((range_AEdAO[1]-range_AEdAO[0])/ 0.3),
             'PdD':   int((range_PdD[1]-range_PdD[0])    / 0.3)}
LIST_SIZE

{'D': 3, 'AEdAO': 2, 'PdD': 3}

In [25]:
t = [LIST_SIZE[v] for v in LIST_SIZE]
all_iter = t[0]*t[1]*t[2]
all_iter

18

In [5]:
save_file = True

In [6]:
def append_to_file(filename, row):
    with open(filename, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(row)
    
def append_to_file_order(filename, D='', AEdAO='', PdD='', Z='', P_B='', n='', fitness='', i=''):
    row = [D, AEdAO, PdD, Z, P_B, n, fitness, i]
    append_to_file(filename, row)
    
def create_file(text):
    filename = dir_name+'/' + text +'.csv'
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        header = ["D = propeller diameter [m]", 
                  "AEdAO = expanded area ratio", 
                  "PdD = pitch ratio", 
                  "Z = propeller's number of blades", 
                  "P_B = power brake", 
                  "n = Propeller angular speed [rpm]", 
                  "fitness", 
                  "i = thread pool id"]
        writer.writerow(header) 
    return filename

def create_dir(text):
    now = datetime.now()
    dir_name = './' + text +'_'+ now.strftime("%Y_%m_%d_%H_%M")
    try:
        mkdir(dir_name)
    except: pass
    return dir_name

In [7]:
def run_octave_evaluation(V_S,D,Z,AEdAO,PdD):
    P_B, n = [0, 0]
    with Oct2Py() as octave:
        octave.warning ("off", "Octave:data-file-in-path");
        octave.addpath('./allCodesOctave');
        P_B, n = octave.F_LabH2(V_S,D,Z,AEdAO,PdD, nout=2)
    return [P_B, n]

In [23]:
def get_P_B(x, i=None):
    D     = x[0]
    AEdAO = x[1]
    PdD   = x[2]
    P_B, n = run_octave_evaluation(V_S,D,Z,AEdAO,PdD)
    if save_file:
        append_to_file(filename, [D, AEdAO, PdD, Z, P_B, n, '',''])
    return P_B    

Logic for the iterations

In [26]:
def run(z):
    global Z
    Z = z
    # create the csv file with the headers
    if save_file:
        global filename
        filename = create_file(str(Z))

    list_D     = np.linspace(range_D[0],     range_D[1],     LIST_SIZE['D'])
    list_AEdAO = np.linspace(range_AEdAO[0], range_AEdAO[1], LIST_SIZE['AEdAO'])
    list_PdD   = np.linspace(range_PdD[0],   range_PdD[1],   LIST_SIZE['PdD'])
    
    counter = 0
    for D in list_D:
        for AEdAO in list_AEdAO:
            for PdD in list_PdD:
                get_P_B((D, AEdAO, PdD))
                counter += 1
                if (counter % 10) == 0:
                    print('Z:',Z, counter,'/',all_iter)
                    
    return Z

parallel

In [27]:
# create dir named 'gradient'
if save_file:
    global dir_name
    dir_name = create_dir('gradient')
    
if __name__ == '__main__':
    with Pool() as pool:
        for result_z in pool.map(run, range(range_Z[0],range_Z[1]+1)):
            print('end of ', result_z)

Z: 5 10 / 18
Z: 4 10 / 18
Z: 2 10 / 18
Z: 3 10 / 18
Z: 6 10 / 18
Z: 7 10 / 18
end of  2
end of  3
end of  4
end of  5
end of  6
end of  7


results